In [ ]:
!pip install dimod
!pip install tqdm
!pip install dwave-neal
!pip install networkx
!pip install matplotlib
!pip install pandas

import numpy as np
import pandas as pd
import random
import json
import time
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.notebook import tqdm
import dimod
import neal
import os
import gc


In [ ]:
# import os
# import glob

# # # # # Paths to delete
# patterns_to_delete = [
#     "benchmark_summary_*.csv",
#     "problem_instance_6p_*.json",
#     "trust_over_time_benchmark_*.csv"
# ]

# deleted_files = []

# for pattern in patterns_to_delete:
#     for file_path in glob.glob(pattern):
#         os.remove(file_path)
#         deleted_files.append(file_path)

# print("✅ Deleted files:")
# for f in deleted_files:
#     print("🗑️", f)

# if not deleted_files:
#     print("No files matched.")

In [ ]:
TOTAL_ROUNDS = 100
NUM_READS = 50
SPOOF_FRAC_RANGE = (0.05, 0.2)
SPOOF_SEVERITY = 0.1
LOCAL_VIEW_RADIUS = 2



In [ ]:
from google.colab import files

print("📁 Upload all 10 benchmark JSON files at once:")
uploaded = files.upload()
benchmark_files = list(uploaded.keys())
print(f"✅ Uploaded files: {benchmark_files}")


📁 Upload all 10 benchmark JSON files at once:


Saving problem_instance_6p_200642.json to problem_instance_6p_200642.json
Saving problem_instance_6p_200643.json to problem_instance_6p_200643.json
Saving problem_instance_6p_200646.json to problem_instance_6p_200646.json
Saving problem_instance_6p_200648.json to problem_instance_6p_200648.json
Saving problem_instance_6p_200650.json to problem_instance_6p_200650.json
Saving problem_instance_6p_200652.json to problem_instance_6p_200652.json
Saving problem_instance_6p_200653.json to problem_instance_6p_200653.json
Saving problem_instance_6p_200655.json to problem_instance_6p_200655.json
Saving problem_instance_6p_200657.json to problem_instance_6p_200657.json
Saving problem_instance_6p_200658.json to problem_instance_6p_200658.json
Saving problem_instance_6p_200659.json to problem_instance_6p_200659.json
Saving problem_instance_6p_200660.json to problem_instance_6p_200660.json
Saving problem_instance_6p_200662.json to problem_instance_6p_200662.json
Saving problem_instance_6p_200663.json

In [ ]:
def build_benchmark_graph(data):
    G = nx.Graph()
    num_tasks = len(data["T_e"])

    for i in range(num_tasks):
        urgency = data["T_e"][i] / 100.0
        x, y = data["task_locations"][i]
        G.add_node(f"T{i}", type="task", urgency=urgency, x=x, y=y, structural_integrity=1.0, occupants=1, defense_level=0.0)

    for i in range(num_tasks):
        for j in range(i + 1, num_tasks):
            dist = data["T_t"][i][j]
            G.add_edge(f"T{i}", f"T{j}", distance=dist, status="open")

    return G, [f"T{i}" for i in range(num_tasks)]

def init_benchmark_robots(data):
    robots = {}
    skill_vectors = data["R"]
    for i, skills in enumerate(skill_vectors):
        trust = round(random.uniform(0.5, 0.95), 2)
        robots[i] = {
            'trust': trust,
            'trust_history': [],
            'compromised': False,
            'success_count': 0,
            'fail_count': 0,
            'assigned_room': f"T{random.randint(0, len(data['T_e']) - 1)}",
            'spoofed_room': f"T{random.randint(0, len(data['T_e']) - 1)}",
            'skills': skills,
            'role': 'worker',
            'initial_trust': trust,
        }
    return robots

def define_precedence_map(prec_list):
    precedence_map = {}
    for succ, pred in prec_list:
        t = f"T{succ}"
        if t not in precedence_map:
            precedence_map[t] = []
        precedence_map[t].append(f"T{pred}")
    return precedence_map

def get_distance_matrix(data):
    dist_matrix = {}
    num_tasks = len(data["T_e"])
    for i in range(num_tasks):
        for j in range(num_tasks):
            dist_matrix[(f"T{i}", f"T{j}")] = data["T_t"][i][j]
    return dist_matrix


In [ ]:
def greedy_assignment(robots, tasks, graph, precedence_map, dist_matrix):
    task_assignments = {task: [] for task in tasks}
    for rid, r in robots.items():
        best_task = None
        best_score = -np.inf
        for t in tasks:
            dist = dist_matrix.get((r['assigned_room'], t), 999)

            if t in precedence_map:
                if any(dep not in [r['assigned_room'] for r in robots.values()] for dep in precedence_map[t]):
                    continue

            urgency = graph.nodes[t].get('urgency', 0.2)
            trust = r['trust']
            integrity = graph.nodes[t].get('structural_integrity', 1.0)
            skill = r['skills'][int(t[1:]) % len(r['skills'])]

            score = trust * urgency * integrity * skill - dist * 0.01
            if score > best_score:
                best_score = score
                best_task = t

        if best_task:
            r['assigned_room'] = best_task
            task_assignments[best_task].append(rid)
    return task_assignments

def precompute_static_costs(robots, tasks, graph):
    static_terms = {}
    quad = {}
    agent_ids = list(robots.keys())

    for agent_id, robot in robots.items():
        for t_idx, task in enumerate(tasks):
            data = graph.nodes[task]
            skill = robot['skills'][t_idx % len(robot['skills'])]
            base = data.get('structural_integrity', 1.0) * skill
            static_terms[(agent_id, task)] = base - data.get('defense_level', 0.0)

    for task in tasks:
        occ = graph.nodes[task].get('occupants', 1)
        synergy = -0.5 * occ if occ >= 2 else 0
        for i in range(len(agent_ids)):
            for j in range(i + 1, len(agent_ids)):
                pair = ((agent_ids[i], task), (agent_ids[j], task))
                quad[pair] = 10.0 + synergy

    return static_terms, quad

def update_dynamic_costs_np(robots, tasks, graph, static_terms, dist_matrix,
                            dist_penalty=0.02, comp_penalty=1.0, local_radius=20):
    cost_dict = {}
    for agent_id, r in robots.items():
        for t in tasks:
            dist = dist_matrix.get((r['assigned_room'], t), 999)
            if dist > local_radius:
                continue

            key = (agent_id, t)
            base = static_terms.get(key, 1.0)
            trust = r['trust']
            urgency = graph.nodes[t].get('urgency', 0.2)

            cost = -1.0 * trust * urgency * base + dist_penalty * dist
            if r['compromised']:
                cost += comp_penalty
            cost_dict[key] = cost

    if cost_dict:
        min_cost = min(cost_dict.values())
        max_cost = max(cost_dict.values())
        range_cost = max_cost - min_cost + 1e-6
        for k in cost_dict:
            cost_dict[k] = (cost_dict[k] - min_cost) / range_cost
    return cost_dict


def build_or_update_bqm(bqm, cost_dict, quad_penalties):
    if bqm is None:
        bqm = dimod.BinaryQuadraticModel('BINARY')
        for var in cost_dict:
            bqm.add_variable(var, 0.0)
        for (var1, var2), penalty in quad_penalties.items():
            bqm.add_interaction(var1, var2, penalty)
    for var, cost in cost_dict.items():
        bqm.set_linear(var, cost)
    return bqm

def apply_qubo_assignments(result, robots, tasks, precedence_map):
    task_assignments = {task: [] for task in tasks}
    assigned_tasks = set()
    assigned_rooms = set(r['assigned_room'] for r in robots.values())

    for (agent_id, task), val in result.items():
        if val != 1 or task in assigned_tasks:
            continue
        deps = precedence_map.get(task, [])
        if all(dep in assigned_rooms for dep in deps):
            robots[agent_id]['assigned_room'] = task
            task_assignments[task].append(agent_id)
            assigned_tasks.add(task)
        else:
            robots[agent_id]['assigned_room'] = f"T{random.randint(0, len(tasks) - 1)}"
    return task_assignments


In [ ]:
def spoofing_attack(robots, tasks, dist_matrix, spoof_frac_range=(0.05, 0.2), severity=0.1):
    spoof_frac = random.uniform(*spoof_frac_range)
    num_to_spoof = max(1, int(spoof_frac * len(robots)))

    sorted_robots = sorted(robots.items(), key=lambda item: -item[1]['trust'])
    targeted = [rid for rid, _ in sorted_robots[:num_to_spoof]]

    for rid in targeted:
        robots[rid]['compromised'] = True

    for rid, r in robots.items():
        if r['compromised']:
            decoy = random.choice(tasks)
            while decoy == r['assigned_room']:
                decoy = random.choice(tasks)
            r['spoofed_room'] = decoy
            r['trust'] = max(0.05, r['trust'] - severity)

def reset_spoofed_rooms(robots):
    for r in robots.values():
        r['spoofed_room'] = r['assigned_room']

def update_urgency(graph, robots, tasks):
    treated = set(r['spoofed_room'] for r in robots.values())
    for task in tasks:
        curr = graph.nodes[task]['urgency']
        if task in treated:
            graph.nodes[task]['urgency'] = round(max(0.1, curr - 0.05), 2)
        else:
            graph.nodes[task]['urgency'] = round(min(1.0, curr + 0.02), 2)

def run_metrics(robots, task_assignments, tasks, precedence_map, dist_matrix):
    trust_vector = np.array([r['trust'] for r in robots.values()])
    spoofed_flags = np.array([r['trust'] < 0.6 for r in robots.values()])
    num_spoofed = np.sum(spoofed_flags)
    num_recovered = np.sum((trust_vector >= 0.6) & spoofed_flags)

    successful_tasks = sum(1 for agents in task_assignments.values() if len(agents) >= 1)
    total_tasks = len(task_assignments)
    coord_success = successful_tasks / (total_tasks + 1e-6)

    violations = 0
    for task, deps in precedence_map.items():
        if task in task_assignments and deps:
            assigned_rooms = set(r['assigned_room'] for r in robots.values())
            if any(dep not in assigned_rooms for dep in deps):
                violations += len(task_assignments[task])

    duplicate_count = sum(1 for agents in task_assignments.values() if len(agents) > 1)

    avg_dist = np.mean([
        dist_matrix.get((r['assigned_room'], r['spoofed_room']), 50)
        for r in robots.values()
    ])

    trusturgency = sum(r['trust'] * tasks.index(r['assigned_room']) for r in robots.values() if r['assigned_room'] in tasks)

    return {
        "avg_trust": float(np.mean(trust_vector)),
        "avg_spoofed": int(num_spoofed),
        "avg_recovery": int(num_recovered),
        "avg_coord_success": float(coord_success),
        "avg_violations": int(violations),
        "avg_duplicates": int(duplicate_count),
        "avg_distance": float(avg_dist),
        "trusturgency": float(trusturgency)
    }


In [ ]:
def run_benchmark_simulation(file_path, use_qubo=True, total_rounds=100, num_reads=50, local_radius=LOCAL_VIEW_RADIUS):
    with open(file_path, 'r') as f:
        data = json.load(f)

    graph, tasks = build_benchmark_graph(data)
    robots = init_benchmark_robots(data)
    precedence_map = define_precedence_map(data.get("precedence_constraints", []))
    dist_matrix = get_distance_matrix(data)

    static_terms, quad_penalties = precompute_static_costs(robots, tasks, graph)
    bqm = None
    sampler = neal.SimulatedAnnealingSampler()

    trust_over_time = []
    spoofed_total = []
    recovered_count = []
    coordinated_tasks = []
    solve_times = []
    trusturgency_scores = []
    violations = []
    duplicates = []
    distances = []

    for round_num in range(total_rounds):
        reset_spoofed_rooms(robots)

        t0 = time.time()
        if use_qubo:
            cost_dict = update_dynamic_costs_np(
                robots, tasks, graph, static_terms, dist_matrix,
                dist_penalty=0.02, comp_penalty=1.0, local_radius=local_radius * 10
            )
            bqm = build_or_update_bqm(bqm, cost_dict, quad_penalties)
            result = sampler.sample(bqm, num_reads=num_reads).first.sample
            task_assignments = apply_qubo_assignments(result, robots, tasks, precedence_map)
        else:
            task_assignments = greedy_assignment(robots, tasks, graph, precedence_map, dist_matrix)
        solve_times.append(time.time() - t0)

        spoofing_attack(robots, tasks, dist_matrix, spoof_frac_range=SPOOF_FRAC_RANGE, severity=SPOOF_SEVERITY)
        update_urgency(graph, robots, tasks)

        for r in robots.values():
            outcome = random.random() < graph.nodes[r['spoofed_room']].get('structural_integrity', 0.9)
            if outcome:
                r['trust'] = min(0.95, r['trust'] + 0.02)
                r['success_count'] += 1
            else:
                r['trust'] = max(0.05, r['trust'] - 0.05)
                r['fail_count'] += 1
            r['trust_history'].append(r['trust'])

        metrics = run_metrics(robots, task_assignments, tasks, precedence_map, dist_matrix)

        trust_over_time.append(metrics["avg_trust"])
        spoofed_total.append(metrics["avg_spoofed"])
        recovered_count.append(metrics["avg_recovery"])
        coordinated_tasks.append(metrics["avg_coord_success"])
        trusturgency_scores.append(metrics["trusturgency"])
        violations.append(metrics["avg_violations"])
        duplicates.append(metrics["avg_duplicates"])
        distances.append(metrics["avg_distance"])

        CURING_BUDGET = int(len(robots) * 0.2)
        total_cleaner_success = sum(r['success_count'] for r in robots.values())
        comp = [(rid, r) for rid, r in robots.items() if r['compromised']]
        comp.sort(key=lambda x: (-x[1]['trust'], -x[1]['success_count']))
        for rid, r in comp[:CURING_BUDGET]:
            room_data = graph.nodes[r['spoofed_room']]
            p = (
                0.1 +
                0.02 * r['success_count'] +
                0.05 * room_data.get('urgency', 0.5)
            )
            if random.random() < min(1.0, p):
                r['compromised'] = False
                r['trust'] = min(0.95, r['trust'] + 0.05)

        if round_num % 10 == 0:
            gc.collect()

    label = "Benchmark-QUBO" if use_qubo else "Benchmark-Greedy"
    summary = {
        "method": label,
        "file": os.path.basename(file_path),
        "final_trust": np.mean(trust_over_time),
        "trusturgency": np.mean(trusturgency_scores),
        "avg_spoofed": np.mean(spoofed_total),
        "avg_recovery": np.mean(recovered_count),
        "avg_coord_success": np.mean(coordinated_tasks),
        "avg_violations": np.mean(violations),
        "avg_duplicates": np.mean(duplicates),
        "avg_distance": np.mean(distances),
        "avg_solve_time": np.mean(solve_times),
    }

    trust_df = pd.DataFrame({
        "round": list(range(total_rounds)),
        "avg_trust": trust_over_time
    })
    trust_df.to_csv(f"trust_over_time_benchmark_{label.lower()}_{file_path[-6:-5]}.csv", index=False)

    return summary


In [ ]:
all_summaries = []

for file_path in tqdm(benchmark_files, desc="🔁 Running QUBO Benchmarks"):
    result = run_benchmark_simulation(file_path, use_qubo=True)
    all_summaries.append(result)

for file_path in tqdm(benchmark_files, desc="🔁 Running Greedy Benchmarks"):
    result = run_benchmark_simulation(file_path, use_qubo=False)
    all_summaries.append(result)

# 💾 Save all results
df_all = pd.DataFrame(all_summaries)
df_all.to_csv("benchmark_summary_all_raw.csv", index=False)

df_qubo = df_all[df_all['method'] == "Benchmark-QUBO"]
df_greedy = df_all[df_all['method'] == "Benchmark-Greedy"]

df_qubo.mean(numeric_only=True).to_frame().T.to_csv("benchmark_summary_across_qubo.csv", index=False)
df_greedy.mean(numeric_only=True).to_frame().T.to_csv("benchmark_summary_across_greedy.csv", index=False)

print("✅ All benchmark simulations complete and saved!")



🔁 Running QUBO Benchmarks:   0%|          | 0/100 [00:00<?, ?it/s]

🔁 Running Greedy Benchmarks:   0%|          | 0/100 [00:00<?, ?it/s]

✅ All benchmark simulations complete and saved!
